**Prefácio:**

**Implementação baseada no trabalho de:** ...

**Os resultados deste trabalho são apresentados no artigo (se utilizado, consulte o trabalho a seguir):** ...

**Amostras de dados:**
>* Legendas de imagens em inglês da amostra de legendas de imagens do COCO
* Poemas em russo do site stihi.ru

**Descrição do trabalho:**
> Neste trabalho, foram realizados a implementação, o treinamento e o teste de redes neurais para geração de textos curtos aleatórios. Redes neurais como LSTM e SeqGAN são utilizadas. A qualidade da geração de texto foi avaliada com base na métrica BLEU. Os seguintes exemplos de dados são usados ​​para treinamento e teste:
* Legendas de imagens em inglês da coleção COCO Image Captions
* Poemas em russo do site stihi.ru

Lista de todas as bibliotecas instaladas e suas versões

In [ ]:
!pip list

Package                            Version
---------------------------------- -------------------
absl-py                            1.4.0
accelerate                         1.2.1
aiohappyeyeballs                   2.4.4
aiohttp                            3.11.11
aiosignal                          1.3.2
alabaster                          1.0.0
albucore                           0.0.19
albumentations                     1.4.20
altair                             5.5.0
annotated-types                    0.7.0
anyio                              3.7.1
argon2-cffi                        23.1.0
argon2-cffi-bindings               21.2.0
array_record                       0.6.0
arviz                              0.20.0
astropy                            6.1.7
astropy-iers-data                  0.2025.1.20.0.32.27
astunparse                         1.6.3
atpublic                           4.1.0
attrs                              24.3.0
audioread                          3.0.1
autograd           

Conexão no Google Drive

In [ ]:
import torch
from torch.autograd import Variable
from math import ceil

def prepare_generator_batch(samples, start_letter=0, gpu=False):
    """
    Takes samples (a batch) and returns

    Inputs: samples, start_letter, cuda
        - samples: batch_size x seq_len (Tensor with a sample in each row)

    Returns: inp, target
        - inp: batch_size x seq_len (same as target, but with start_letter prepended)
        - target: batch_size x seq_len (Variable same as samples)
    """

    batch_size, seq_len = samples.size()

    inp = torch.zeros(batch_size, seq_len)
    target = samples
    inp[:, 0] = start_letter
    inp[:, 1:] = target[:, :seq_len-1]

    inp = Variable(inp).type(torch.LongTensor)
    target = Variable(target).type(torch.LongTensor)

    if gpu:
        inp = inp.cuda()
        target = target.cuda()

    return inp, target


def prepare_discriminator_data(pos_samples, neg_samples, gpu=False):
    """
    Takes positive (target) samples, negative (generator) samples and prepares inp and target data for discriminator.

    Inputs: pos_samples, neg_samples
        - pos_samples: pos_size x seq_len
        - neg_samples: neg_size x seq_len

    Returns: inp, target
        - inp: (pos_size + neg_size) x seq_len
        - target: pos_size + neg_size (boolean 1/0)
    """

    inp = torch.cat((pos_samples, neg_samples), 0).type(torch.LongTensor)
    target = torch.ones(pos_samples.size()[0] + neg_samples.size()[0])
    target[pos_samples.size()[0]:] = 0

    # shuffle
    perm = torch.randperm(target.size()[0])
    target = target[perm]
    inp = inp[perm]

    inp = Variable(inp)
    target = Variable(target)

    if gpu:
        inp = inp.cuda()
        target = target.cuda()

    return inp, target


def batchwise_sample(gen, num_samples, batch_size):
    """
    Sample num_samples samples batch_size samples at a time from gen.
    Does not require gpu since gen.sample() takes care of that.
    """

    samples = []
    for i in range(int(ceil(num_samples/float(batch_size)))):
        samples.append(gen.sample(batch_size))

    return torch.cat(samples, 0)[:num_samples]


def batchwise_oracle_nll(gen, oracle, num_samples, batch_size, max_seq_len, start_letter=0, gpu=False):
    s = batchwise_sample(gen, num_samples, batch_size)
    oracle_nll = 0
    for i in range(0, num_samples, batch_size):
        inp, target = prepare_generator_batch(s[i:i+batch_size], start_letter, gpu)
        oracle_loss = oracle.batchNLLLoss(inp, target) / max_seq_len
        oracle_nll += oracle_loss.data.item()

    return oracle_nll/(num_samples/batch_size)

In [ ]:
import os

path = '/content/'
os.chdir(path)

Declarando parâmetros padrão

In [ ]:
from __future__ import print_function
from math import ceil
import numpy as np
import sys

import torch
import torch.optim as optim
import torch.nn as nn

import torch.autograd as autograd
import torch.nn.functional as F
import torch.nn.init as init

DATASET = 'coco_image_captions'  # nome do conjunto de dados, lista de conjuntos de dados: ['stihi_ru', 'coco_image_captions']
CUDA = True  # uso do cuda (gpu/tpu): True/False
BATCH_SIZE = 1000  # número de exemplos por lote/batch
MLE_TRAIN_EPOCHS = 30  # número de épocas ao treinar o gerador usando MLE (máxima verossimilhança)
DIS_TRAIN_ITERATIONS = 300  # número de iterações ao treinar o discriminador (iterações contêm várias épocas, em cada iteração os dados de treinamento são atualizados)
DIS_TRAIN_EPOCHS = 1  # número de épocas em uma iteração do treinamento do discriminador
ADV_TRAIN_EPOCHS = 25  # número de épocas ao treinar o gerador usando aprendizado adversarial/aprendizado por reforço
POS_NEG_SAMPLES = 10000

GEN_EMBEDDING_DIM = 150  # largura da camada de Embedding do gerador
GEN_HIDDEN_DIM = 150  # número de neurônios nas camadas do gerador
DIS_EMBEDDING_DIM = 150  # largura da camada de Embedding do discriminador
DIS_HIDDEN_DIM = 150  # número de neurônios nas camadas do discriminador

# os parâmetros START_LETTER, VOCAB_SIZE, MAX_SEQ_LEN e FILE_PATHS são definidos automaticamente abaixo
START_LETTER = None  # palavra inicial (a palavra que é dada primeiro à rede neural do gerador)
MAX_SEQ_LEN = None  # comprimento do exemplo gerado
VOCAB_SIZE = None  # número de palavras no vocabulário
FILE_PATHS = None  # caminhos para os arquivos do conjunto de dados
CLOSING_WORD = None  # palavra final / palavra substituta que é colocada no final da sentença

# seleção do conjunto de dados e inicialização dos parâmetros globais correspondentes
if DATASET == 'stihi_ru':  # conjunto de dados com poesias russas do site stihi_ru (comprimento do exemplo 10 palavras, tamanho do vocabulário 5000 palavras)
  START_LETTER = 0
  MAX_SEQ_LEN = 10
  VOCAB_SIZE = 5000
  CLOSING_WORD = 4999  # 4997
  FILE_PATHS = {'train': r'/content/stihi_ru_realtrain_cotra.txt', 'test': r'/content/stihi_ru_realtest_coco.txt',
                'vocab': r'/content/stihi_ru_vocab_cotra.pkl', 'saved_models': r'/content/'}
elif DATASET == 'coco_image_captions':  # conjunto de dados COCO Image Captions (comprimento do exemplo 20 palavras, tamanho do vocabulário 4980 palavras)
  START_LETTER = 0
  MAX_SEQ_LEN = 20
  VOCAB_SIZE = 4838
  CLOSING_WORD = 1814
  FILE_PATHS = {'train': r'/content/coco_image_captions_train.txt', 'test': r'/content/coco_image_captions_test.txt',
                'vocab': r'/content/coco_image_captions_vocab_cotra_test.pkl', 'saved_models': r'/content/'}

Inicialização de geradores de números aleatórios.

In [ ]:
# inicialização de geradores de números aleatórios.
torch.random.manual_seed(50)
np.random.seed(50)
# aleatoriedade do sistema!!!

Declaração de classe do gerador

In [ ]:
class Generator(nn.Module):

  def __init__(self, embedding_dim, hidden_dim, vocab_size, max_seq_len, gpu=False):
    super(Generator, self).__init__()
    self.hidden_dim = hidden_dim # número de elementos na camada oculta
    self.embedding_dim = embedding_dim # tamanho da camada de incorporação
    self.max_seq_len = max_seq_len # comprimento dos exemplos gerados
    self.vocab_size = vocab_size # tamanho do dicionário usado para geração
    self.gpu = gpu # uso de cuda (Verdadeiro/Falso)
    self.lstm_num_layers = 1 # número de camadas LSTM

    self.embeddings = nn.Embedding(vocab_size, embedding_dim) # declaração de camada de incorporação
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=self.lstm_num_layers) # Declaração de camada LSTM
    self.lstm2out = nn.Linear(hidden_dim, vocab_size) # declaração da camada de saída

  def init_hidden(self, batch_size=1):
    # inicialização do estado da camada LSTM
    h = autograd.Variable(torch.zeros(self.lstm_num_layers, batch_size, self.hidden_dim))
    c = autograd.Variable(torch.zeros(self.lstm_num_layers, batch_size, self.hidden_dim))

    if self.gpu:
        return h.cuda(), c.cuda()
    else:
        return h, c

  def forward(self, inp, hidden, c):
    """
    Embeds input and applies LSTM one token at a time (seq_len = 1)
    """
    # input dim                                             # batch_size
    emb = self.embeddings(inp)                              # batch_size x embedding_dim
    emb = emb.view(1, -1, self.embedding_dim)               # 1 x batch_size x embedding_dim
    out, (hidden, c) = self.lstm(emb, (hidden, c))                    # 1 x batch_size x hidden_dim (out)
    out = self.lstm2out(out.view(-1, self.hidden_dim))       # batch_size x vocab_size
    out = F.log_softmax(out, dim=1)
    return out, hidden, c

  def sample(self, num_samples, start_letter=0, degree=1):
    """
    Samples the network and returns num_samples samples of length max_seq_len.

    Outputs: samples, hidden
        - samples: num_samples x max_seq_length (a sampled sequence in each row)
    """

    samples = torch.zeros(num_samples, self.max_seq_len).type(torch.LongTensor)

    h, c = self.init_hidden(num_samples)
    inp = autograd.Variable(torch.LongTensor([start_letter]*num_samples))
    if self.gpu:
        samples = samples.cuda()
        inp = inp.cuda()

    for i in range(self.max_seq_len):
        out, h, c = self.forward(inp, h, c)               # out: num_samples x vocab_size
        out = torch.exp(out)**degree
        out = torch.multinomial(out, 1)  # num_samples x 1 (sampling from each row)
        samples[:, i] = out.view(-1).data

        inp = out.view(-1)

    return samples

  def batchNLLLoss(self, inp, target):
    """
    Returns the NLL Loss for predicting target sequence.

    Inputs: inp, target
        - inp: batch_size x seq_len
        - target: batch_size x seq_len

        inp should be target with <s> (start letter) prepended
    """

    loss_fn = nn.NLLLoss()
    batch_size, seq_len = inp.size()
    inp = inp.permute(1, 0)           # seq_len x batch_size
    target = target.permute(1, 0)     # seq_len x batch_size
    h, c = self.init_hidden(batch_size)

    loss = 0
    for i in range(seq_len):
        out, h, c = self.forward(inp[i], h, c)
        loss += loss_fn(out, target[i])

    return loss     # per batch

  def batchPGLoss(self, inp, target, reward):
    """
    Returns a pseudo-loss that gives corresponding policy gradients (on calling .backward()).
    Inspired by the example in http://karpathy.github.io/2016/05/31/rl/

    Inputs: inp, target
        - inp: batch_size x seq_len
        - target: batch_size x seq_len
        - reward: batch_size (discriminator reward for each sentence, applied to each token of the corresponding
                  sentence)

        inp should be target with <s> (start letter) prepended
    """

    batch_size, seq_len = inp.size()
    inp = inp.permute(1, 0)          # seq_len x batch_size
    target = target.permute(1, 0)    # seq_len x batch_size
    h, c = self.init_hidden(batch_size)

    loss = 0
    for i in range(seq_len):
        out, h, c = self.forward(inp[i], h, c)
        # TODO: should h be detached from graph (.detach())?
        for j in range(batch_size):
            loss += -out[j][target.data[i][j]]*reward[j]#     # log(P(y_t|Y_1:Y_{t-1})) * Q

    return loss/batch_size

Declarando a classe Discriminator

In [ ]:
class Discriminator(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, max_seq_len, gpu=False, dropout=0.2):
        super(Discriminator, self).__init__()
        self.hidden_dim = hidden_dim # tamanho da camada oculta
        self.embedding_dim = embedding_dim # largura da camada de incorporação
        self.max_seq_len = max_seq_len # comprimento do exemplo de entrada
        self.gpu = gpu # uso de cuda (Verdadeiro/Falso)

        self.embeddings = nn.Embedding(vocab_size, embedding_dim) # Declaração de camada de incorporação
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=3, bidirectional=True, dropout=dropout) # Declaração de camada LSTM

        self.lstm2hidden = nn.Linear(2*3*hidden_dim, hidden_dim) # declaração de camada oculta
        # self.lstm2hidden = nn.Linear(max_seq_len*hidden_dim*2, hidden_dim) # declaração de camada oculta

        self.dropout_linear = nn.Dropout(p=dropout)
        self.hidden2out = nn.Linear(hidden_dim, 1) # declaração da camada de saída

    def init_hidden(self, batch_size):
        # inicialização de camadas LSTM
        h = autograd.Variable(torch.zeros(2*3*1, batch_size, self.hidden_dim))
        c = autograd.Variable(torch.zeros(2*3*1, batch_size, self.hidden_dim))

        if self.gpu:
            return h.cuda(), c.cuda()
        else:
            return h, c

    def forward(self, input, hidden, c):
        # input dim                                                # batch_size x seq_len
        # print(input.shape)
        emb = self.embeddings(input)                               # batch_size x seq_len x embedding_dim
        # print(emb.shape)
        emb = emb.permute(1, 0, 2)                                 # seq_len x batch_size x embedding_dim
        # print(emb.shape)
        out_lstm, (hidden, c) = self.lstm(emb, (hidden, c))                          # 4 x batch_size x hidden_dim

        hidden = hidden.permute(1, 0, 2).contiguous()              # batch_size x 4 x hidden_dim
        # hidden = out_lstm
        # print(hidden.shape, hidden.view(-1, self.max_seq_len*self.hidden_dim).shape)
        # print(hidden.permute(1, 0, 2).contiguous().shape, hidden.permute(1, 0, 2).contiguous().view(-1, self.max_seq_len*self.hidden_dim*2).shape)

        out = self.lstm2hidden(hidden.view(-1, 6*self.hidden_dim))  # batch_size x 4*hidden_dim
        # out = self.lstm2hidden(hidden.view(-1, self.max_seq_len*self.hidden_dim*2))  # batch_size x 4*hidden_dim

        out = torch.relu(out)
        # out = out * torch.sigmoid(0.1 * out) # функция активации Swish: x * sigmoid(b*x)
        out = self.dropout_linear(out)
        out = self.hidden2out(out)                                 # batch_size x 1
        out = torch.sigmoid(out)
        return out

    def batchClassify(self, inp):
        """
        Classifies a batch of sequences.

        Inputs: inp
            - inp: batch_size x seq_len

        Returns: out
            - out: batch_size ([0,1] score)
        """

        h, c = self.init_hidden(inp.size()[0])
        out = self.forward(inp, h, c)
        return out.view(-1)

    def batchBCELoss(self, inp, target):
        """
        Returns Binary Cross Entropy Loss for discriminator.

         Inputs: inp, target
            - inp: batch_size x seq_len
            - target: batch_size (binary 1/0)
        """

        loss_fn = nn.BCELoss()
        h, c = self.init_hidden(inp.size()[0])
        out = self.forward(inp, h, c)
        return loss_fn(out, target)

Declarando funções que retornam dados

In [ ]:
# função de geração de dados reais
def sampler_example(batch_size):
  x = data_file_train[np.random.randint(0, len(data_file_train), batch_size)]
  y = np.concatenate([x[:, 1:], np.zeros([batch_size, 1])+VOCAB_SIZE-2], axis=-1)
  return x, y

# função de geração de dados reais
def sampler_example_test(batch_size):
  x = data_file_test[np.random.randint(0, len(data_file_test), batch_size)]
  y = np.concatenate([x[:, 1:], np.zeros([batch_size, 1])+VOCAB_SIZE-2], axis=-1)
  return x, y

Funções de treinamento do gerador

In [ ]:
# Função de aprendizagem do gerador baseada em MLE
def train_generator_MLE(gen, gen_opt, real_samples_train, real_samples_test, epochs):
    """
    Max Likelihood Pretraining for the generator
    """
    for epoch in range(epochs):
        print('epoch %d : ' % (epoch + 1), end='')
        sys.stdout.flush()
        total_loss = 0

        # treinamento
        for i in range(0, len(real_samples_train), BATCH_SIZE):
            inp_train, target_train = prepare_generator_batch(real_samples_train[i:i + BATCH_SIZE], start_letter=START_LETTER,
                                                          gpu=CUDA)
            gen_opt.zero_grad()
            loss = gen.batchNLLLoss(inp_train, target_train)
            loss.backward()
            gen_opt.step()

            total_loss += loss.data.item()

            if (i / BATCH_SIZE) % ceil(
                            ceil(len(real_samples_train) / float(BATCH_SIZE)) / 10.) == 0:  # roughly every 10% of an epoch
                print('.', end='')
                sys.stdout.flush()

        # each loss in a batch is loss per sample
        total_loss = total_loss / ceil(len(real_samples_train) / float(BATCH_SIZE)) / MAX_SEQ_LEN
        print(' average_train_NLL = %.4f' % total_loss, end='')

        # testando
        test_loss = 0

        for i in range(0, len(real_samples_test), BATCH_SIZE):
            inp_test, target_test = prepare_generator_batch(real_samples_test[i:i + BATCH_SIZE], start_letter=START_LETTER,
                                                              gpu=CUDA)
            loss = gen.batchNLLLoss(inp_test, target_test)
            test_loss += loss.data.item()
        test_loss = test_loss / ceil(len(real_samples_test) / float(BATCH_SIZE)) / MAX_SEQ_LEN
        print(' average_test_NLL = %.4f' % test_loss)

def test_mle(gen, real_samples_train, real_samples_test):
  '''
  Testando o gerador em amostras de treinamento e teste.
  '''
  # testando no treinamento
  test_loss = 0
  for i in range(0, len(real_samples_train), BATCH_SIZE):
      inp_test, target_test = prepare_generator_batch(real_samples_train[i:i + BATCH_SIZE], start_letter=START_LETTER,
                                                        gpu=CUDA)
      loss = gen.batchNLLLoss(inp_test, target_test)
      test_loss += loss.data.item()
  test_loss = test_loss / ceil(len(real_samples_train) / float(BATCH_SIZE)) / MAX_SEQ_LEN
  print('average_train_NLL = %.4f' % test_loss, end='')

  # testando em teste
  test_loss = 0
  for i in range(0, len(real_samples_test), BATCH_SIZE):
      inp_test, target_test = prepare_generator_batch(real_samples_test[i:i + BATCH_SIZE], start_letter=START_LETTER,
                                                        gpu=CUDA)
      loss = gen.batchNLLLoss(inp_test, target_test)
      test_loss += loss.data.item()
  test_loss = test_loss / ceil(len(real_samples_test) / float(BATCH_SIZE)) / MAX_SEQ_LEN
  print(' average_test_NLL = %.4f' % test_loss)

# função de aprendizagem do gerador baseada em aprendizagem por reforço RL
def train_generator_PG(gen, gen_opt, dis, num_batches): # !renomear!
    """
    The generator is trained using policy gradients, using the reward from the discriminator.
    Training is done for num_batches batches.
    """

    for batch in range(num_batches):
        s = gen.sample(BATCH_SIZE*4)        # 64 works best
        inp, target = prepare_generator_batch(s, start_letter=START_LETTER, gpu=CUDA)
        rewards = dis.batchClassify(target)

        gen_opt.zero_grad()
        pg_loss = gen.batchPGLoss(inp, target, rewards)
        pg_loss.backward()
        gen_opt.step()
    print()

Função de aprendizagem discriminadora

In [ ]:
# função de aprendizagem discriminadora
def train_discriminator(discriminator, dis_opt, real_data_samples, generator, d_steps, epochs):
    """
    Training the discriminator on real_data_samples (positive) and generated samples from generator (negative).
    Samples are drawn d_steps times, and the discriminator is trained for epochs epochs.
    """

    # generating a small validation set before training
    pos_val = real_data_samples[np.random.randint(0, len(real_data_samples), 500)] #sampler_example(250)
    neg_val = generator.sample(500)
    val_inp, val_target = prepare_discriminator_data(pos_val, neg_val, gpu=CUDA)

    for d_step in range(d_steps):
        s = batchwise_sample(generator, POS_NEG_SAMPLES, BATCH_SIZE)
        dis_inp, dis_target = prepare_discriminator_data(real_data_samples, s, gpu=CUDA)
        val_pred = discriminator.batchClassify(val_inp)
        print('ANTES DO TREINO: val_acc = %.4f' % (
            torch.sum((val_pred>0.5)==(val_target>0.5)).data.item()/1000.))
        for epoch in range(epochs):
            print('d-step %d epoch %d : ' % (d_step + 1, epoch + 1), end='')
            sys.stdout.flush()
            total_loss = 0
            total_acc = 0

            for i in range(0, 2 * POS_NEG_SAMPLES, BATCH_SIZE):
                inp, target = dis_inp[i:i + BATCH_SIZE], dis_target[i:i + BATCH_SIZE]
                dis_opt.zero_grad()
                out = discriminator.batchClassify(inp)
                loss_fn = nn.BCELoss()
                loss = loss_fn(out, target)
                loss.backward()
                dis_opt.step()

                total_loss += loss.data.item()
                total_acc += torch.sum((out>0.5)==(target>0.5)).data.item()

                if (i / BATCH_SIZE) % ceil(ceil(2 * POS_NEG_SAMPLES / float(
                        BATCH_SIZE)) / 10.) == 0:  # roughly every 10% of an epoch
                    print('.', end='')
                    sys.stdout.flush()

            total_loss /= ceil(2 * POS_NEG_SAMPLES / float(BATCH_SIZE))
            total_acc /= float(2 * POS_NEG_SAMPLES)

            val_pred = discriminator.batchClassify(val_inp)
            print(' average_loss = %.4f, train_acc = %.4f, val_acc = %.4f' % (
                total_loss, total_acc, torch.sum((val_pred>0.5)==(val_target>0.5)).data.item()/1000.))

Função de avaliação da qualidade da geração de texto

In [ ]:
# avaliação de qualidade pela métrica BLEU
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.bleu_score import sentence_bleu
from datetime import datetime
import nltk
import random
from scipy import stats

# função para avaliar a qualidade da geração de texto usando a métrica BLEU
def BLEU(reference_sample, test_sample, print_iteration=100, flag_print=True):
  if flag_print:
    print("--- --- ---\nStart BLEU")
  pad = CLOSING_WORD
  #################################################
  reference = []
  for line in reference_sample:
    candidate = []
    for i in line:
      if i == pad:
        break
      candidate.append(i)

    reference.append(candidate)
  #################################################
  hypothesis_list_leakgan = []
  for line in test_sample:
    while line[-1] == str(pad):
      line.remove(str(pad))
    hypothesis_list_leakgan.append(line)
  #################################################
  random.shuffle(hypothesis_list_leakgan)
  #################################################

  smoothing_function = SmoothingFunction().method1

  mass_bleu = []
  for ngram in range(2,6):
      weight = tuple((1. / ngram for _ in range(ngram)))
      bleu_leakgan = []
      bleu_supervise = []
      bleu_base2 = []
      num = 0
      for h in hypothesis_list_leakgan:
          BLEUscore = nltk.translate.bleu_score.sentence_bleu(reference, h, weight, smoothing_function = smoothing_function)
          num += 1
          bleu_leakgan.append(BLEUscore)

          if num%print_iteration == 0 and flag_print:
            print(ngram, num, sum(bleu_leakgan)/len(bleu_leakgan))

      mass_bleu.append(1.0 * sum(bleu_leakgan) / len(bleu_leakgan))
      if flag_print:
        print('--- --- ---')
        print(len(weight), '-gram BLEU score : ', 1.0 * sum(bleu_leakgan) / len(bleu_leakgan), "\n")
  return mass_bleu

Função de salvamento de modelo

In [ ]:
# função para salvar modelos (serialização: modelos geradores e discriminadores, parâmetros padrão, dados de treinamento)
def save_models(data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer, name):
  state = {
      'default_parameters': {'VOCAB_SIZE': VOCAB_SIZE, 'MAX_SEQ_LEN': MAX_SEQ_LEN, 'GEN_EMBEDDING_DIM': GEN_EMBEDDING_DIM,
                             'GEN_HIDDEN_DIM': GEN_HIDDEN_DIM, 'DIS_EMBEDDING_DIM': DIS_EMBEDDING_DIM, 'DIS_HIDDEN_DIM': DIS_HIDDEN_DIM},
      'data_file_tensor_train': data_file_tensor_train,
      'gen_state_dict': gen.state_dict(),
      'dis_state_dict': dis.state_dict(),
      'gen_optimizer': gen_optimizer.state_dict(),
      'dis_optimizer': dis_optimizer.state_dict(),
  }
  torch.save(state, name)

Função de carregamento do modelo

In [ ]:
# função para carregar modelos (desserializar: modelos geradores e discriminadores, parâmetros padrão, dados de treinamento)
def load_models(name):
  if CUDA:
    device = torch.device('cuda')
  else:
    device = torch.device('cpu')

  print('state')
  state = torch.load(name, map_location=device)

  print('default_parameters')
  VOCAB_SIZE = state['default_parameters']['VOCAB_SIZE']
  MAX_SEQ_LEN = state['default_parameters']['MAX_SEQ_LEN']
  GEN_EMBEDDING_DIM = state['default_parameters']['GEN_EMBEDDING_DIM']
  GEN_HIDDEN_DIM = state['default_parameters']['GEN_HIDDEN_DIM']
  DIS_EMBEDDING_DIM = state['default_parameters']['DIS_EMBEDDING_DIM']
  DIS_HIDDEN_DIM = state['default_parameters']['DIS_HIDDEN_DIM']

  print('data_file_tensor_train')
  data_file_tensor_train = torch.tensor(state['data_file_tensor_train'])

  print('Generator')
  gen = Generator(GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)
  gen.load_state_dict(state['gen_state_dict'])
  gen_optimizer = optim.Adam(gen.parameters(), lr=0.001)
  gen_optimizer.load_state_dict(state['gen_optimizer'])

  print('Discriminator')
  dis = Discriminator(DIS_EMBEDDING_DIM, DIS_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)
  dis.load_state_dict(state['dis_state_dict'])
  dis_optimizer = optim.Adagrad(dis.parameters())
  dis_optimizer.load_state_dict(state['dis_optimizer'])

  print('CUDA')
  if CUDA:
    data_file_tensor_train = data_file_tensor_train.cuda()
    gen = gen.cuda()
    dis = dis.cuda()

  return [data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
          VOCAB_SIZE, MAX_SEQ_LEN, GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, DIS_EMBEDDING_DIM, DIS_HIDDEN_DIM]

Carregando o conjunto de dados

In [ ]:
# carregando dicionário
import pickle

vocab_file = FILE_PATHS['vocab']
word, vocab = pickle.load(open(vocab_file, 'rb'))

In [ ]:
# carregamento da amostra de treinamento
f = open(FILE_PATHS['train'], 'r')
data_file_train = []
for line in f:
  line = line.replace('\n', '')
  line = line.split()
  for i in range(len(line)):
    line[i] = int(line[i])
  data_file_train.append(line)
data_file_train = np.array(data_file_train)[:, :MAX_SEQ_LEN]
print("Exemplos na amostra de treinamento: ", len(data_file_train))

Exemplos na amostra de treinamento:  80000


In [ ]:
# carregamento da amostra de teste
f = open(FILE_PATHS['test'], 'r')
data_file_test = []
for line in f:
  line = line.replace('\n', '')
  line = line.split()
  for i in range(len(line)):
    line[i] = int(line[i])
  data_file_test.append(line)
data_file_test = np.array(data_file_test)[:, :MAX_SEQ_LEN]
print("Exemplos na amostra de teste: ", len(data_file_test))

Exemplos na amostra de teste:  5000


In [ ]:
# exemplos da amostra de treinamento
print("Exemplos da amostra de treinamento")
samples = sampler_example(50)[0]
output_function = []
for samp in samples:
  line = [word[x] for x in samp]
  line = ' '.join(line)
  output_function.append(line)

for i, output in enumerate(output_function):
  print("#", i, "\tExemplo: ", output)

Exemplos da amostra de treinamento
# 0 	Exemplo:  A street in front of an airport with people going in the entrance .            
# 1 	Exemplo:  Blue and green bird sitting on the ground next to plants .                
# 2 	Exemplo:  A racer is sitting on a horse in the gates .                  
# 3 	Exemplo:  A man skiing a jump , while in the background there is snow melting off the slopes .    
# 4 	Exemplo:  A man that is holding a knife and a pot with broccoli .              
# 5 	Exemplo:  A bed in a bedroom that has lots of things sitting on top of it .        
# 6 	Exemplo:  A skateboarder does a handstand trick on a skate - ramp .                
# 7 	Exemplo:  A tall white building with three vehicles parked in front of it .              
# 8 	Exemplo:  A man admiring a black and yellow motorcycle on a white background .              
# 9 	Exemplo:  A cup of soup , a sandwich and a sweet roll on a table .          
# 10 	Exemplo:  Traffic on a busy city road with street ligh

Avaliação da qualidade dos exemplos do conjunto de treinamento baseado no BLEU (para comparação com o gerador)

In [ ]:
%%time
# Avaliação de qualidade com base nos exemplos da amostra de treinamento (para comparação com o gerador)
print("Avaliação dos exemplos da amostra de treinamento com base no BLEU")
BLEU(data_file_test.tolist(), data_file_train[:500].tolist(), print_iteration=100)

Avaliação dos exemplos da amostra de treinamento com base no BLEU
--- --- ---
Start BLEU
2 100 0.5668908859860812
2 200 0.5565824398207067
2 300 0.5492119198714641
2 400 0.5467045446976672
2 500 0.5499117864293881
--- --- ---
2 -gram BLEU score :  0.5499117864293881 

3 100 0.45817138194805873
3 200 0.44736460910970616
3 300 0.4359582265418135
3 400 0.4304990035415631
3 500 0.43699142793369117
--- --- ---
3 -gram BLEU score :  0.43699142793369117 

4 100 0.34424180007897254
4 200 0.32889664912071476
4 300 0.3152189334404297
4 400 0.3095685622316223
4 500 0.3173224414085977
--- --- ---
4 -gram BLEU score :  0.3173224414085977 

5 100 0.23686041045461245
5 200 0.22011932885266372
5 300 0.20931541371110826
5 400 0.20581532744507264
5 500 0.21083373415979104
--- --- ---
5 -gram BLEU score :  0.21083373415979104 

CPU times: user 5min 44s, sys: 673 ms, total: 5min 45s
Wall time: 5min 56s


[0.5499117864293881,
 0.43699142793369117,
 0.3173224414085977,
 0.21083373415979104]

Criação de redes neurais do gerador e discriminador

In [ ]:
# Declaração do gerador e discriminador de redes neurais, preparação de amostras de dados para uso pelo pytorch
gen = Generator(GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)
dis = Discriminator(DIS_EMBEDDING_DIM, DIS_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)

if CUDA:
  gen = gen.cuda()
  dis = dis.cuda()
  data_file_tensor_train = torch.tensor(data_file_train).cuda()
  data_file_tensor_test = torch.tensor(data_file_test).cuda()
else:
  data_file_tensor_train = torch.tensor(data_file_train)
  data_file_tensor_test = torch.tensor(data_file_test)

gen_optimizer = optim.Adam(gen.parameters(), lr=0.001) # lr=0.001
dis_optimizer = optim.Adagrad(dis.parameters())

Treinando um gerador com base em MLE

In [ ]:
%%time
test_mle(gen, data_file_tensor_train, data_file_tensor_test)

average_train_NLL = 8.5446 average_test_NLL = 8.5451
CPU times: user 7.94 s, sys: 115 ms, total: 8.05 s
Wall time: 8.12 s


In [ ]:
%%time
# Treinamento de gerador baseado em MLE / pré-treinamento de gerador
print('Iniciando o treinamento do gerador baseado em MLE...')
gen_optimizer = optim.Adam(gen.parameters()) # lr=0.0002
train_generator_MLE(gen, gen_optimizer, data_file_tensor_train, data_file_tensor_test, MLE_TRAIN_EPOCHS) # MLE_TRAIN_EPOCHS

test_mle(gen, data_file_tensor_train, data_file_tensor_test)
# salvando o resultado da aprendizagem
save_models(data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
            FILE_PATHS['saved_models'] + r'/' + r'seqgan_mle.pytorch')
# epoch 1 : .......... average_train_NLL = 1.8447 average_test_NLL = 1.9937

Iniciando o treinamento do gerador baseado em MLE...
epoch 1 : .......... average_train_NLL = 4.5138 average_test_NLL = 3.3173
epoch 2 : .......... average_train_NLL = 3.0475 average_test_NLL = 2.9099
epoch 3 : .......... average_train_NLL = 2.7634 average_test_NLL = 2.7012
epoch 4 : .......... average_train_NLL = 2.5905 average_test_NLL = 2.5820
epoch 5 : .......... average_train_NLL = 2.4759 average_test_NLL = 2.4898
epoch 6 : .......... average_train_NLL = 2.3922 average_test_NLL = 2.4348
epoch 7 : .......... average_train_NLL = 2.3276 average_test_NLL = 2.3860
epoch 8 : .......... average_train_NLL = 2.2767 average_test_NLL = 2.3403
epoch 9 : .......... average_train_NLL = 2.2326 average_test_NLL = 2.3062
epoch 10 : .......... average_train_NLL = 2.1932 average_test_NLL = 2.2736
epoch 11 : .......... average_train_NLL = 2.1576 average_test_NLL = 2.2472
epoch 12 : .......... average_train_NLL = 2.1273 average_test_NLL = 2.2134
epoch 13 : .......... average_train_NLL = 2.0967 average

In [ ]:
%%time
test_mle(gen, data_file_tensor_train, data_file_tensor_test)

average_train_NLL = 1.8009 average_test_NLL = 1.9609
CPU times: user 7.4 s, sys: 14 ms, total: 7.42 s
Wall time: 7.42 s


Gerando textos de amostra com base em MLE

In [ ]:
# # exemplos de textos gerados
print("Exemplos de textos gerados com base em MLE")
degree = 1
print("Degree:", degree)
samples = gen.sample(50, degree=degree).cpu().detach().numpy()

output_function = []
for i, samp in enumerate(samples):
  line = [word[x] for x in samp]
  line = ' '.join(line)
  output_function.append(line)
  bleu = BLEU(data_file_test.tolist(), [samp], flag_print=False)
  print("#", i, "\tExemplo: ", line, ' '*(100-len(line)), '\tAvaliação: ', bleu)

Exemplos de textos gerados com base em MLE
Degree: 1
# 0 	Exemplo:  The train traveling down the track on town track just set out of the park .                           	Avaliação:  [0.5803810000880093, 0.4539847774172143, 0.1531682455208201, 0.08078132896612913]
# 1 	Exemplo:  The fence sitting inside the bathroom surrounded by trees and tube .                                  	Avaliação:  [0.41675437673324534, 0.268229131353767, 0.10322080150552766, 0.05891010340986106]
# 2 	Exemplo:  A train moving tracks next to a wooden size platform in the country in a dimly environment .          	Avaliação:  [0.6529286250990105, 0.4558685532582043, 0.2732231510487644, 0.12834826432879642]
# 3 	Exemplo:  A woman on the tennis court preparing to hit with a bat .                                             	Avaliação:  [0.4893604849295929, 0.34174500988297, 0.22012248449054683, 0.1079711005739161]
# 4 	Exemplo:  A cat is sitting on a train along with many other riders ' s figurine .              

Avaliação da qualidade da geração de texto após treinamento usando MLE

In [ ]:
%%time
print("Avaliação da qualidade da geração de texto baseada em BLEU após treinamento com MLE")
# controle de qualidade da educação
BLEU(data_file_test.tolist(), gen.sample(500, degree=degree).cpu().detach().numpy().tolist(), print_iteration=100)

Avaliação da qualidade da geração de texto baseada em BLEU após treinamento com MLE
--- --- ---
Start BLEU
2 100 0.5293642270728148
2 200 0.5177477282229925
2 300 0.5187759811123885
2 400 0.5224344367661522
2 500 0.5226959797212712
--- --- ---
2 -gram BLEU score :  0.5226959797212712 

3 100 0.3745155103422013
3 200 0.3747070946116354
3 300 0.3766445189439405
3 400 0.3824035755361252
3 500 0.3826083717484991
--- --- ---
3 -gram BLEU score :  0.3826083717484991 

4 100 0.23614238711588037
4 200 0.2383679075019982
4 300 0.24110822943618232
4 400 0.24672461169817064
4 500 0.24742168834243194
--- --- ---
4 -gram BLEU score :  0.24742168834243194 

5 100 0.13042908649105356
5 200 0.14142368359462995
5 300 0.14374599352648856
5 400 0.14743754287257302
5 500 0.1487098461666625
--- --- ---
5 -gram BLEU score :  0.1487098461666625 

CPU times: user 5min 49s, sys: 778 ms, total: 5min 49s
Wall time: 5min 52s


[0.5226959797212712,
 0.3826083717484991,
 0.24742168834243194,
 0.1487098461666625]

Estimativa com exponenciação

In [ ]:
# exemplos de textos gerados
print("Exemplos de textos gerados com base em MLE")
degree = 1.5
print("Degree:", degree)
samples = gen.sample(50, degree=degree).cpu().detach().numpy()

output_function = []
for i, samp in enumerate(samples):
  line = [word[x] for x in samp]
  line = ' '.join(line)
  output_function.append(line)
  bleu = BLEU(data_file_test.tolist(), [samp], flag_print=False)
  print("#", i, "\tExemplo:", line, ' '*(100-len(line)), '\tAvaliação: ', bleu)

Exemplos de textos gerados com base em MLE
Degree: 1.5
# 0 	Exemplo: The man is doing the trick on a court near a man .                                                    	Avaliação:  [0.5548826333938605, 0.37160681631842096, 0.13181065657459776, 0.07163684030886805]
# 1 	Exemplo: A man holding up a donut in a bowl of flowers in a room .                                             	Avaliação:  [0.716350399411379, 0.6109741448719966, 0.404727200247809, 0.17575428316242378]
# 2 	Exemplo: A woman in a dress eating a pizza with a slice missing .                                              	Avaliação:  [0.5548826333938605, 0.4090062242255915, 0.2518750835198491, 0.12026082240959093]
# 3 	Exemplo: A green train down train station on the tracks near a mountain .                                      	Avaliação:  [0.5231483637805969, 0.3572998474390311, 0.22759483012793272, 0.11089345578518284]
# 4 	Exemplo: A lady is holding a tennis racket on a clay court .                                   

In [ ]:
%%time
print("Avaliação da qualidade da geração de texto baseada em BLEU após treinamento com MLE")
# controle de qualidade da educação
BLEU(data_file_test.tolist(), gen.sample(500, degree=degree).cpu().detach().numpy().tolist(), print_iteration=100)

Avaliação da qualidade da geração de texto baseada em BLEU após treinamento com MLE
--- --- ---
Start BLEU
2 100 0.5513005877690402
2 200 0.5529753989584019
2 300 0.5592306944409703
2 400 0.5595382814981565
2 500 0.5601664492111109
--- --- ---
2 -gram BLEU score :  0.5601664492111109 

3 100 0.4606382121619048
3 200 0.4618684565582859
3 300 0.46878096063362346
3 400 0.47007795098469557
3 500 0.47195595945252483
--- --- ---
3 -gram BLEU score :  0.47195595945252483 

4 100 0.3517695060056129
4 200 0.3559383287399828
4 300 0.3613050667937825
4 400 0.3618204616411319
4 500 0.3652760221418807
--- --- ---
4 -gram BLEU score :  0.3652760221418807 

5 100 0.23378122392990164
5 200 0.24110267354895967
5 300 0.24716420870391498
5 400 0.24830012621073533
5 500 0.25284693317639867
--- --- ---
5 -gram BLEU score :  0.25284693317639867 

CPU times: user 5min 43s, sys: 732 ms, total: 5min 43s
Wall time: 5min 46s


[0.5601664492111109,
 0.47195595945252483,
 0.3652760221418807,
 0.25284693317639867]

Pré-treinamento do discriminador

In [ ]:
%%time
# pré-treinamento do discriminador
print('Iniciando o treinamento do discriminador...')
# dis_optimizer = optim.Adagrad(dis.parameters()) # , lr=0.0001
# dis_optimizer = optim.Adam(gen.parameters(), lr=0.001, weight_decay=1e-5)#0.001
# dis_optimizer = optim.Adadelta(gen.parameters())
# dis_optimizer = optim.Adagrad(dis.parameters())#, lr=0.0001)#, weight_decay=1e-5)
#, weight_decay=1e-5) # regularização
train_discriminator(dis, dis_optimizer, data_file_tensor_train, gen, DIS_TRAIN_ITERATIONS, DIS_TRAIN_EPOCHS)# 25, 1 | (15, 3), (25, 1)

# salvando o resultado da aprendizagem
save_models(data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
            FILE_PATHS['saved_models'] + r'/' + r'seqgan_pretraining_dis.pytorch')

# ANTES DO TREINO: val_acc = 0.5230
# d-step 50 epoch 1 : .......... average_loss = 0.2859, train_acc = 0.9136, val_acc = 0.5240

Iniciando o treinamento do discriminador...
ANTES DO TREINO: val_acc = 0.5000
d-step 1 epoch 1 : .......... average_loss = 0.4396, train_acc = 0.8496, val_acc = 0.5000
ANTES DO TREINO: val_acc = 0.5000
d-step 2 epoch 1 : .......... average_loss = 0.3491, train_acc = 0.8884, val_acc = 0.5000
ANTES DO TREINO: val_acc = 0.5000
d-step 3 epoch 1 : .......... average_loss = 0.3403, train_acc = 0.8904, val_acc = 0.5000
ANTES DO TREINO: val_acc = 0.5000
d-step 4 epoch 1 : .......... average_loss = 0.3445, train_acc = 0.8877, val_acc = 0.5000
ANTES DO TREINO: val_acc = 0.5000
d-step 5 epoch 1 : .......... average_loss = 0.3305, train_acc = 0.8912, val_acc = 0.5000
ANTES DO TREINO: val_acc = 0.5000
d-step 6 epoch 1 : .......... average_loss = 0.3381, train_acc = 0.8879, val_acc = 0.5000
ANTES DO TREINO: val_acc = 0.5000
d-step 7 epoch 1 : .......... average_loss = 0.3319, train_acc = 0.8864, val_acc = 0.5000
ANTES DO TREINO: val_acc = 0.5000
d-step 8 epoch 1 : .......... average_loss = 0.3292, t

Aprendizagem competitiva. Treinamento de um gerador com base no aprendizado por reforço

In [ ]:
test_mle(gen, data_file_tensor_train, data_file_tensor_test)
# average_train_NLL = 1.7516 average_test_NLL = 1.9832

average_train_NLL = 1.8009 average_test_NLL = 1.9609


In [ ]:
%%time
# gen_optimizer = optim.Adagrad(gen.parameters(), lr=0.0005)#Adam #, lr=0.0005
# dis_optimizer = optim.Adagrad(dis.parameters())
# gen_optimizer = optim.Adam(gen.parameters(), lr=0.0001)#Adam #, lr=0.0005
# dis_optimizer = optim.Adagrad(dis.parameters(), lr=0.001)#, lr=0.002

# Gerador de Aprendizagem Adversarial
print('\nStarting Adversarial Training...')

for epoch in range(ADV_TRAIN_EPOCHS):# ADV_TRAIN_EPOCHS
    print('\n--------\nEPOCH %d\n--------' % (epoch+1))
    # treinamento de gerador
    print('\nAdversarial Training Generator : ', end='')
    train_generator_PG(gen, gen_optimizer, dis, 1)

    # teste nll
    print('\nTesting Generator : ', end='')
    test_mle(gen, data_file_tensor_train, data_file_tensor_test)

    # treinamento discriminador
    print('\nAdversarial Training Discriminator : ')
    train_discriminator(dis, dis_optimizer, data_file_tensor_train, gen, 5, 1)#3, 1

# salvando o resultado da aprendizagem
save_models(data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
            FILE_PATHS['saved_models'] + r'/' + r'seqgan_adversarial_training.pytorch')
# epoch 3 : .......... average_train_NLL = 1.8005 average_test_NLL = 1.9824
# average_train_NLL = 1.7340 average_test_NLL = 2.0599


Starting Adversarial Training...

--------
EPOCH 1
--------

Adversarial Training Generator : 

Testing Generator : average_train_NLL = 1.8014 average_test_NLL = 1.9605

Adversarial Training Discriminator : 
ANTES DO TREINO: val_acc = 0.8150
d-step 1 epoch 1 : .......... average_loss = 0.1510, train_acc = 0.9581, val_acc = 0.7910
ANTES DO TREINO: val_acc = 0.7940
d-step 2 epoch 1 : .......... average_loss = 0.1592, train_acc = 0.9550, val_acc = 0.8220
ANTES DO TREINO: val_acc = 0.8250
d-step 3 epoch 1 : .......... average_loss = 0.1639, train_acc = 0.9538, val_acc = 0.8170
ANTES DO TREINO: val_acc = 0.8220
d-step 4 epoch 1 : .......... average_loss = 0.1607, train_acc = 0.9536, val_acc = 0.8160
ANTES DO TREINO: val_acc = 0.8110
d-step 5 epoch 1 : .......... average_loss = 0.1597, train_acc = 0.9552, val_acc = 0.8200

--------
EPOCH 2
--------

Adversarial Training Generator : 

Testing Generator : average_train_NLL = 1.8020 average_test_NLL = 1.9602

Adversarial Training Discriminator

Testando o gerador no conjunto de treinamento após o treinamento adversário.

In [ ]:
%%time
test_mle(gen, data_file_tensor_train, data_file_tensor_test)
# average_train_NLL = 1.5186 average_test_NLL = 2.0730
# average_train_NLL = 1.4459 average_test_NLL = 1.9926

average_train_NLL = 1.8009 average_test_NLL = 1.9540
CPU times: user 7.41 s, sys: 10.5 ms, total: 7.42 s
Wall time: 7.42 s


Gerando textos de amostra com base no SeqGAN

In [ ]:
# exemplos de textos gerados
print("Exemplos de textos gerados com base no SeqGAN")
degree = 1
print("Degree:", degree)
samples = gen.sample(50, degree=degree).cpu().detach().numpy()

output_function = []
for i, samp in enumerate(samples):
  line = [word[x] for x in samp]
  line = ' '.join(line)
  output_function.append(line)
  bleu = BLEU(data_file_test.tolist(), [samp], flag_print=False)
  print("#", i, "\tExemplo: ", line, ' '*(100-len(line)), '\tAvaliação: ', bleu)

Exemplos de textos gerados com base no SeqGAN
Degree: 1
# 0 	Exemplo:  The street scene filled with broccoli all and a box of dessert .                                      	Avaliação:  [0.435285750066007, 0.21915874168443503, 0.0887067909105404, 0.05218449657024714]
# 1 	Exemplo:  A tabby cat sitting on top of a table full of tongs .                                                 	Avaliação:  [0.5619514869490163, 0.5196828406118894, 0.4507482544534971, 0.37822261873373586]
# 2 	Exemplo:  A young girl flying three kite in the kitchen to look .                                               	Avaliação:  [0.435285750066007, 0.27612271191669263, 0.10549074689987335, 0.059944245366591306]
# 3 	Exemplo:  A line of people playing a game with their square ' s leg in the water .                              	Avaliação:  [0.6688560540599386, 0.4208926658419206, 0.2573453280190883, 0.12234579126368764]
# 4 	Exemplo:  A lone dog on the couch against a toilet window above brush .                  

Avaliação da qualidade da geração com base no SeqGAN

In [ ]:
%%time
print("Avaliação da qualidade da geração de texto baseada em BLEU após treinamento com SeqGAN")
degree = 1
print("Degree:", degree)
# controle de qualidade da educação
BLEU(data_file_test.tolist(), gen.sample(500, degree=degree).cpu().detach().numpy().tolist(), print_iteration=100)

Avaliação da qualidade da geração de texto baseada em BLEU após treinamento com SeqGAN
Degree: 1
--- --- ---
Start BLEU
2 100 0.5093269360756356
2 200 0.5090302788246426
2 300 0.5096131363695509
2 400 0.5157009129314349
2 500 0.5124317667785041
--- --- ---
2 -gram BLEU score :  0.5124317667785041 

3 100 0.37824655498814763
3 200 0.37499114212674145
3 300 0.3777895329420967
3 400 0.38550795051567127
3 500 0.3824308100492639
--- --- ---
3 -gram BLEU score :  0.3824308100492639 

4 100 0.25843798349028135
4 200 0.2555358919958465
4 300 0.25651910617853324
4 400 0.2624024319143163
4 500 0.26082058408289294
--- --- ---
4 -gram BLEU score :  0.26082058408289294 

5 100 0.1614757084763551
5 200 0.15864465507729197
5 300 0.16054825967869626
5 400 0.16473342785941336
5 500 0.16414649785572483
--- --- ---
5 -gram BLEU score :  0.16414649785572483 

CPU times: user 5min 50s, sys: 777 ms, total: 5min 50s
Wall time: 5min 53s


[0.5124317667785041,
 0.3824308100492639,
 0.26082058408289294,
 0.16414649785572483]

Estimativa com exponenciação

In [ ]:
# exemplos de textos gerados
print("Exemplos de textos gerados com base em SeqGAN com exponenciação")
degree = 1.5
print("Degree:", degree)
samples = gen.sample(50, degree=degree).cpu().detach().numpy()

output_function = []
for i, samp in enumerate(samples):
  line = [word[x] for x in samp]
  line = ' '.join(line)
  output_function.append(line)
  bleu = BLEU(data_file_test.tolist(), [samp], flag_print=False)
  print("#", i, "\tExemplo: ", line, ' '*(100-len(line)), '\tAvaliação: ', bleu)

Exemplos de textos gerados com base em SeqGAN com exponenciação
Degree: 1.5
# 0 	Exemplo:  A man takes a picture of a slice on top of a table .                                                  	Avaliação:  [0.6649099662043687, 0.6046423508984526, 0.5492704080363613, 0.4906806884641922]
# 1 	Exemplo:  A woman standing next to a man on a table with a kite .                                               	Avaliação:  [0.6920602346769048, 0.6639509405734763, 0.6092033169091078, 0.5097976264749036]
# 2 	Exemplo:  A boy is jumping in the air holding onto his surfboard .                                              	Avaliação:  [0.5026246899500345, 0.347892817097899, 0.22308576866161578, 0.10913234729182814]
# 3 	Exemplo:  A man standing in front of a TV with a laptop .                                                       	Avaliação:  [0.5619514869490163, 0.5196828406118894, 0.4507482544534971, 0.40062241498668816]
# 4 	Exemplo:  A group of uniformed people standing next to each other .      

In [ ]:
%%time
print("Avaliação da qualidade da geração de texto baseada em BLEU após treinamento com SeqGAN")
# controle de qualidade da educação
BLEU(data_file_test.tolist(), gen.sample(500, degree=degree).cpu().detach().numpy().tolist(), print_iteration=100)

Avaliação da qualidade da geração de texto baseada em BLEU após treinamento com SeqGAN
--- --- ---
Start BLEU
2 100 0.5549611905245981
2 200 0.5656522528298974
2 300 0.566306108313174
2 400 0.5655395303058187
2 500 0.5645866166564775
--- --- ---
2 -gram BLEU score :  0.5645866166564775 

3 100 0.49315955473500955
3 200 0.5038688095408669
3 300 0.5032140960425149
3 400 0.5015572251494816
3 500 0.49865083503454527
--- --- ---
3 -gram BLEU score :  0.49865083503454527 

4 100 0.413040595201514
4 200 0.4194801251658673
4 300 0.4171087254822016
4 400 0.41547211693344827
4 500 0.4119848242411157
--- --- ---
4 -gram BLEU score :  0.4119848242411157 

5 100 0.3122351335296429
5 200 0.31678152978945984
5 300 0.3141256740663774
5 400 0.3117888980796234
5 500 0.307158973269923
--- --- ---
5 -gram BLEU score :  0.307158973269923 

CPU times: user 5min 41s, sys: 688 ms, total: 5min 42s
Wall time: 5min 44s


[0.5645866166564775,
 0.49865083503454527,
 0.4119848242411157,
 0.307158973269923]

Carregando um modelo salvo

In [ ]:
# carregando modelos
[data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
VOCAB_SIZE, MAX_SEQ_LEN, GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, DIS_EMBEDDING_DIM,
 DIS_HIDDEN_DIM] = load_models(FILE_PATHS['saved_models'] + r'/' + r'seqgan_adversarial_training.pytorch')#[seqgan_mle, seqgan_pretraining_dis, seqgan_adversarial_training]

if(CUDA):
  gen = gen.cuda()
  dis = dis.cuda()
  data_file_tensor_train = torch.tensor(data_file_tensor_train).cuda()
  data_file_tensor_test = torch.tensor(data_file_test).cuda()

state
default_parameters
data_file_tensor_train
Generator
Discriminator
CUDA


<ipython-input-13-0c1e57d4af32>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(name, map_location=device)
<ipython-input-13-0c1e57d4af32>:20: UserWarnin

Carregando um modelo salvo

In [ ]:
degree = 1.5

In [ ]:
# plágio do conjunto de treinamento
samples = gen.sample(1500, degree=degree).cpu().detach().numpy().tolist()
# samples = data_file_test.tolist()[:500]
train_samples = data_file_train.tolist()
n = 0
for i in range(len(samples)):
  if samples[i] in train_samples:
    n += 1
  if i%(len(samples)//10) == 0:
    print(i/len(samples)*100, "%")
print("O número de exemplos gerados que corresponderam à amostra de treinamento: ", n, "de", len(samples))
print("Plágio: ", n/len(samples)*100, "%")

0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
O número de exemplos gerados que corresponderam à amostra de treinamento:  10 de 1500
Plágio:  0.6666666666666667 %


In [ ]:
# originalidade dos exemplos
N_samples = 1500
samples_1 = gen.sample(N_samples, degree=degree).cpu().detach().numpy().tolist()
samples_2 = gen.sample(N_samples, degree=degree).cpu().detach().numpy().tolist()

n = 0
for i in range(len(samples_1)):
  if samples_1[i] in samples_2:
    n += 1
  if i%(len(samples_1)//10) == 0:
    print(i/len(samples_1)*100, "%")

print("Originalidade dos exemplos gerados")
print("O número de exemplos da amostra 1 correspondeu aos exemplos da amostra 2: ", n, "de", len(samples_1))
print("Plágio: ", n/len(samples_1)*100, "%")
print("Originalidade: ", (1-n/len(samples_1))*100, "%")

0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
Originalidade dos exemplos gerados
O número de exemplos da amostra 1 correspondeu aos exemplos da amostra 2:  24 de 1500
Plágio:  1.6 %
Originalidade:  98.4 %


In [ ]:
# originalidade dos exemplos abordagem 2
N_samples = 1500
samples = gen.sample(N_samples, degree=degree).cpu().detach().numpy().tolist()

i = 0
n = 0
while len(samples)>0:
  if samples[0] not in samples[1:]:
    n += 1
  if i%(N_samples//10) == 0:
    print(i/N_samples*100, "%")
  del(samples[0])
  i += 1

print("Originalidade dos exemplos gerados")
print("Exemplos originais gerados: ", n, "de", N_samples)
print("Originalidade: ", n/N_samples*100, "%")
print("Plágio: ", (1-n/N_samples)*100, "%")

0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
Originalidade dos exemplos gerados
Exemplos originais gerados:  1490 de 1500
Originalidade:  99.33333333333333 %
Plágio:  0.666666666666671 %
